# Business Registry Data Exploratory Data Analysis

In [1]:
import pandas as pd
import csv
pd.options.display.max_columns = 999

In [2]:
b_entity = pd.read_csv('../data/business/BizEntity.txt',
                       sep='\t',
                       encoding='latin',
                       engine='python',
                       quoting=csv.QUOTE_NONE)

In [3]:
b_addr = pd.read_csv('../data/business/BizEntityAddress.txt',
                     sep='\t',
                     encoding='latin',
                     engine='python',
                     quoting=csv.QUOTE_NONE)

In [4]:
entity_cols = ['BusinessId', 'BusinessName', 'BusinessTypeDesc',
               'ForeignState', 'ForeignCountry', 'EntityStatus',
               'Good Standing', 'NAICSCODE']
addr_cols = ['BusinessId','StreetAddress1', 'StreetAddress2',
             'City', 'State', 'Zip', 'Country']

### Create a merged df called registry with the useful columns from the separate business registry files.

Contains Business Name and Address information.

In [5]:
registry = b_entity[entity_cols].merge(b_addr[addr_cols],
                                       how='left',
                                       on='BusinessId')

### Create a column called Adrno to speed up fuzzy matching

In [6]:
registry['Adrno'] = registry['StreetAddress1'].str.extract('(\d+)')

In [7]:
registry['Adrno'].head(5)

0     310
1    4426
2    3014
3    1309
4      36
Name: Adrno, dtype: object

There are some address numbers that are NaN because companies write "One" instead of 1, or they did not provide an address number

In [8]:
registry[registry['Adrno'].isna()].head(3)

,BusinessId,BusinessName,BusinessTypeDesc,ForeignState,ForeignCountry,EntityStatus,Good Standing,NAICSCODE,StreetAddress1,StreetAddress2,City,State,Zip,Country,Adrno
100,101,VERIZON WIRELESS (VAW) LLC,Foreign Limited Liability Company,Delaware,NaN,Withdrawn,0,NaN,One Verizon Way,NaN,Basking Ridge,NJ,07920,United States,NaN
116,117,"PRESIDIO TECHNOLOGY CAPITAL, LLC",Domestic Limited Liability Company,NaN,NaN,Active/Compliance,1,NaN,TWO SUN CT,NaN,NORCROSS,GA,30092,United States,NaN
147,148,THE MARTENSON LAW FIRM LLP,Domestic Limited Partnership,NaN,NaN,Void,0,NaN,NaN,NaN,NaN,NaN,NaN,United States,NaN


In [9]:
len(registry[registry['Adrno'].isna()].index)

121939

### Investigate Zip column since that will also be used to speed up fuzzy matching

How many businesses in the zipcodes with the most registrants? This will let us know if we need to narrow down the search space even further from zip...  

With fuzzy matching, each value in the search space (e.g. properties owned by an owner with that zip) needs to be compared with zip in the registry data.

In [10]:
registry.groupby(['Zip'])['Zip'] \
        .count() \
        .sort_values(ascending=False) \
        .head(10)

Zip
30339    38854
30328    36234
30305    34206
30309    31651
30024    31598
30076    30182
30318    30169
30096    30093
30043    27792
30004    27282
Name: Zip, dtype: int64

Some zips have a high number of registrants, meaning comparing an owner to owners with that matching zip will take a while. We may need to narrow it down more.

### Eliminate any improper zip codes

View the kinds of improper zip codes and counts.

In [11]:
registry[(registry['Zip'].str.contains('^[a-z]') == True)] \
        .groupby('Zip')['Zip'] \
        .count() \
        .sort_values(ascending=False)

Zip
ga       2
and a    1
billm    1
f4v3b    1
ga 30    1
j0h 1    1
j4y 0    1
ky1-1    1
l6p1y    1
q        1
regis    1
sw153    1
v        1
w1t 4    1
xv8ms    1
Name: Zip, dtype: int64

NA values for Zip

In [12]:
len(registry[registry['Zip'].isna()].index)

99135

### Percent of Entries with Incorrect or Missing Zip

In [13]:
tot = len(registry[(registry['Zip'].str.contains('^[a-z]') == True) \
        | (registry['Zip'].isna())].index)

tot / len(registry.index) * 100

3.076139820080987

### Delete Entries with Bad Zips

In [14]:
registry = registry[(registry['Zip'].str.contains('^[a-z]') == False) | (registry['Zip'].notna())]

### Delete Remaining Entries with Empty StreetAddress

In [15]:
init = len(registry.index)
registry = registry[registry['StreetAddress1'].notna()]
init - len(registry.index)

4900

In [16]:
len(registry.index)

3119193

### Check if there multiple business registered under the same address

In [17]:
registry.groupby(['StreetAddress1'])['StreetAddress1'].count().sort_values(ascending=False).loc[lambda x : x > 1].head(10)

StreetAddress1
300 Colonial Center Parkway STE 100N    4840
300 Colonial Center Parkway             1207
NO ADDR.                                1182
1209 N ORANGE ST                         638
1700 Northside Drive                     578
3900 Crown Rd SW #16709                  545
3350 Riverwood Parkway                   529
500 SUGAR MILL ROAD                      517
2859 Paces Ferry Road SE                 507
1465 Northside Drive NW                  462
Name: StreetAddress1, dtype: int64

### Percent of Businesses Registered Under an Address with > 1 Business

In [18]:
dup_addr = registry.groupby(['StreetAddress1'])['StreetAddress1'].count().sort_values(ascending=False).loc[lambda x : x > 1].sum()

In [19]:
dup_addr

1352774

In [20]:
dup_addr / len(registry.index) * 100

43.36935867706808

### Finding: there are many businesses registered at the same address; will need to use name for matching as well.  
Additionally, some addresses in the business registry data have slight variations in capitalization, abbreviations, etc.

Drop NaN BusinessNames (3 total)

In [21]:
registry = registry.loc[registry['BusinessName'].notna()]

In [22]:
len(registry.index)

3119190

### Look at some individual cases to see how subsidaries are represented in the data

In [23]:
registry.loc[registry['BusinessName'].str.contains("Invitation Homes")]

,BusinessId,BusinessName,BusinessTypeDesc,ForeignState,ForeignCountry,EntityStatus,Good Standing,NAICSCODE,StreetAddress1,StreetAddress2,City,State,Zip,Country,Adrno
1794844,1808195,Invitation Homes Realty LLC,Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,"1717 Main Street, Suite 2000",NaN,Dallas,TX,75201,United States,1717
2493978,2611760,Invitation Homes Realty Inc,Foreign Profit Corporation,Delaware,NaN,Withdrawn,0,Any legal purpose,1717 Main Street Suite 2000,NaN,Dallas,TX,75201,United States,1717


### Do subsidaries share the same address as the parent corp? Invitation Homes and Progress Residential appear to, although Progress Residential has minor inconsistency in their StreetAddress.

In [24]:
registry.loc[registry['StreetAddress1'] == '1717 Main Street, Suite 2000']

,BusinessId,BusinessName,BusinessTypeDesc,ForeignState,ForeignCountry,EntityStatus,Good Standing,NAICSCODE,StreetAddress1,StreetAddress2,City,State,Zip,Country,Adrno
1794844,1808195,Invitation Homes Realty LLC,Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,"1717 Main Street, Suite 2000",NaN,Dallas,TX,75201,United States,1717
1843180,1856558,"IH3 PROPERTY GEORGIA, L.P.",Foreign Limited Partnership,Delaware,USA,Active/Compliance,1,NaN,"1717 Main Street, Suite 2000",NaN,Dallas,TX,75201,United States,1717
1850123,1863507,2013-1 IH BORROWER G.P. LLC,Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,"1717 Main Street, Suite 2000",NaN,Dallas,TX,75201,United States,1717
1872678,1886086,2017-1 IH Borrower G.P. LLC,Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,"1717 Main Street, Suite 2000",NaN,Dallas,TX,75201,United States,1717
1872747,1886155,2017-1 IH Borrower L.P.,Foreign Limited Partnership,Delaware,USA,Active/Compliance,1,NaN,"1717 Main Street, Suite 2000",NaN,Dallas,TX,75201,United States,1717
1948571,1962039,2014-2 IH BORROWER G.P. LLC,Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,"1717 Main Street, Suite 2000",NaN,Dallas,TX,75201,United States,1717
1965791,1979268,2014-3 IH BORROWER G.P. LLC,Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,"1717 Main Street, Suite 2000",NaN,Dallas,TX,75201,United States,1717
1990917,2004416,2015-1 IH2 BORROWER L.P.,Foreign Limited Partnership,Delaware,USA,Active/Compliance,1,NaN,"1717 Main Street, Suite 2000",NaN,Dallas,TX,75201,United States,1717
1990929,2004428,2015-1 IH2 BORROWER G.P. LLC,Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,"1717 Main Street, Suite 2000",NaN,Dallas,TX,75201,United States,1717
2025607,2039132,2015-2 IH2 Borrower G.P. LLC,Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,"1717 Main Street, Suite 2000",NaN,Dallas,TX,75201,United States,1717


In [25]:
registry.loc[registry['BusinessName'].str.contains("Progress Residential")]

,BusinessId,BusinessName,BusinessTypeDesc,ForeignState,ForeignCountry,EntityStatus,Good Standing,NAICSCODE,StreetAddress1,StreetAddress2,City,State,Zip,Country,Adrno
1969323,1982800,"Progress Residential Borrower 5, LLC",Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,7500 N. Dobson Rd.,Suite 300,Scottsdale,AZ,85256,United States,7500
2008026,2021539,"Progress Residential Borrower 7, LLC",Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,7500 N. Dobson Rd.,Suite 300,Scottsdale,AZ,85256,United States,7500
2045349,2058886,"Progress Residential Borrower 9, LLC",Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,7500 N. Dobson Road,Suite 300,Scottsdale,AZ,85256,United States,7500
2097296,2115452,"Progress Residential Borrower 12, LLC",Foreign Limited Liability Company,Delaware,NaN,Active/Compliance,1,Any legal purpose,7500 N. Dobson Road,Suite 300,Scottsdale,AZ,85256,United States,7500
2310782,2361205,"Progress Residential Borrower 1, LLC",Foreign Limited Liability Company,Delaware,NaN,Active/Compliance,1,Any legal purpose,7500 N. Dobson Rd,Suite 300,Scottsdale,AZ,85256,United States,7500
2358349,2417707,"Progress Residential Borrower 2, LLC",Foreign Limited Liability Company,Delaware,NaN,Active/Compliance,1,Any legal purpose,7500 N. Dobson Rd,Suite 300,Scottsdale,AZ,85256,United States,7500
2390884,2456761,"Progress Residential Borrower 3, LLC",Foreign Limited Liability Company,Delaware,NaN,Active/Compliance,1,Any legal purpose,7500 N. Dobson Rd,Suite 300,Scottsdale,AZ,85256,United States,7500
2414830,2485457,"Progress Residential Management Services, LLC",Foreign Limited Liability Company,Delaware,NaN,Active/Compliance,1,Any legal purpose,7500 N. Dobson Rd,Suite 300,Scottsdale,AZ,85256,United States,7500
2464572,2546104,"Progress Residential Borrower 4,LLC",Foreign Limited Liability Company,Delaware,NaN,Active/Compliance,1,Any legal purpose,7500 N. Dobson Rd,Suite 300,Scottsdale,AZ,85256,United States,7500
2493097,2610722,"Progress Residential Bridge Borrower, LLC",Foreign Limited Liability Company,Delaware,NaN,Active/Compliance,1,Any legal purpose,7500 NORTH DOBSON ROAD,SUITE 300,SCOTTSDALE,AZ,85256,United States,7500


In [26]:
registry.loc[registry['BusinessName'].str.contains("AMERICAN HOMES 4")]

,BusinessId,BusinessName,BusinessTypeDesc,ForeignState,ForeignCountry,EntityStatus,Good Standing,NAICSCODE,StreetAddress1,StreetAddress2,City,State,Zip,Country,Adrno
1682114,1694514,"AMERICAN HOMES 4 RENT, LLC",Foreign Limited Liability Company,Delaware,NaN,Withdrawn,0,NaN,630 Trade Center Drive,NaN,Las Vegas,NV,89119,United States,630
1683930,1696351,"AMERICAN HOMES 4 RENT MANAGEMENT, LLC",Foreign Limited Liability Company,Georgia,USA,Dissolved,0,NaN,"22917 Pacific Coast Highway, Ste 300",NaN,Malibu,CA,90265,NaN,22917
1743391,1756447,"RJ AMERICAN HOMES 4 RENT ONE, LLC",Foreign Limited Liability Company,Delaware,NaN,Revoked,0,NaN,"30601 Agoura Road, Suite 200L",NaN,Agoura Hills,CA,91301,NaN,30601
1750069,1763202,"RJ AMERICAN HOMES 4 RENT TWO, LLC",Foreign Limited Liability Company,Delaware,NaN,Active/Compliance,1,NaN,"23975 Park Sorrento, Suite 300",NaN,Calabasas,CA,91302,United States,23975
1783173,1796523,"AMERICAN HOMES 4 RENT MANAGEMENT HOLDINGS, LLC",Foreign Limited Liability Company,Georgia,USA,Active/Compliance,1,NaN,23975 PARK SORRENTO STE 300,NaN,Calabasas,CA,91302,United States,23975
1807793,1821148,"AMERICAN HOMES 4 RENT PROPERTIES FOUR, LLC",Foreign Limited Liability Company,Delaware,NaN,Withdrawn,0,NaN,30601 Agoura Road,Suite 200L,Agoura Hills,CA,91301,United States,30601
1807794,1821149,"AMERICAN HOMES 4 RENT PROPERTIES TWO, LLC",Foreign Limited Liability Company,Delaware,NaN,Withdrawn,0,NaN,30601 Agoura Road,Suite 200L,Agoura Hills,CA,91301,United States,30601
1808134,1821489,"AMERICAN HOMES 4 RENT, L.P. (DE)",Foreign Limited Partnership,Delaware,USA,Withdrawn,0,NaN,"22917 PACIFIC COAST HIGHWAY, SUITE 300",NaN,Malibu,CA,90265,NaN,22917
1808561,1821916,"AMERICAN HOMES 4 RENT PROPERTIES ONE, LLC",Foreign Limited Liability Company,Delaware,NaN,Active/Compliance,1,NaN,23975 PARK SORRENTO STE 300,NaN,Calabasas,CA,91302,United States,23975
1850234,1863618,"AMERICAN HOMES 4 RENT PROPERTIES EIGHT, LLC",Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,23975 Park Sorrento,Suite 300,Calabasas,CA,91302,United States,23975


In [27]:
registry.loc[registry['StreetAddress1'] == '30601 Agoura Road']

,BusinessId,BusinessName,BusinessTypeDesc,ForeignState,ForeignCountry,EntityStatus,Good Standing,NAICSCODE,StreetAddress1,StreetAddress2,City,State,Zip,Country,Adrno
1690278,1702763,"AH4R-GA 2, LLC",Foreign Limited Liability Company,Delaware,NaN,Withdrawn,0,NaN,30601 Agoura Road,Suite 200L,Agoura Hills,CA,91301,United States,30601
1728857,1741756,"AH4R I GA, LLC",Foreign Limited Liability Company,Delaware,NaN,Withdrawn,0,NaN,30601 Agoura Road,Suite 200L,Agoura Hills,CA,91301,United States,30601
1807793,1821148,"AMERICAN HOMES 4 RENT PROPERTIES FOUR, LLC",Foreign Limited Liability Company,Delaware,NaN,Withdrawn,0,NaN,30601 Agoura Road,Suite 200L,Agoura Hills,CA,91301,United States,30601
1807794,1821149,"AMERICAN HOMES 4 RENT PROPERTIES TWO, LLC",Foreign Limited Liability Company,Delaware,NaN,Withdrawn,0,NaN,30601 Agoura Road,Suite 200L,Agoura Hills,CA,91301,United States,30601
1911951,1925391,"AMH 2014-1 EQUITY OWNER, LLC",Foreign Limited Liability Company,Delaware,USA,Withdrawn,0,NaN,30601 Agoura Road,Suite 200L,Agoura Hills,CA,91301,United States,30601
1940366,1953826,SFR 2014-GA LLC,Foreign Limited Liability Company,Delaware,USA,Withdrawn,0,NaN,30601 Agoura Road,Suite 200L,Agoura Hills,CA,91301,United States,30601
2000333,2013845,"AH4R-GA, LLC",Foreign Limited Liability Company,Delaware,USA,Withdrawn,0,NaN,30601 Agoura Road,Suite 200L,Agoura Hills,CA,91301,United States,30601
2000344,2013856,"AH4R-GA 3, LLC",Foreign Limited Liability Company,Delaware,USA,Withdrawn,0,NaN,30601 Agoura Road,Suite 200L,Agoura Hills,CA,91301,United States,30601
2000351,2013863,"AH4R - GA 4, LLC",Foreign Limited Liability Company,Delaware,USA,Withdrawn,0,NaN,30601 Agoura Road,Suite 200L,Agoura Hills,CA,91301,United States,30601


In [28]:
registry.loc[registry['StreetAddress1'] == '630 Trade Center Drive']

,BusinessId,BusinessName,BusinessTypeDesc,ForeignState,ForeignCountry,EntityStatus,Good Standing,NAICSCODE,StreetAddress1,StreetAddress2,City,State,Zip,Country,Adrno
1682114,1694514,"AMERICAN HOMES 4 RENT, LLC",Foreign Limited Liability Company,Delaware,NaN,Withdrawn,0,NaN,630 Trade Center Drive,NaN,Las Vegas,NV,89119,United States,630
1909493,1922930,"American Residential Properties TRS, LLC",Foreign Limited Liability Company,Delaware,USA,Withdrawn,0,NaN,630 Trade Center Drive,NaN,Las Vegas,NV,89119,United States,630
2095702,2113663,"ARP MANAGEMENT GEORGIA, LLC",Domestic Limited Liability Company,Georgia,NaN,Terminated,0,Any legal purpose,630 Trade Center Drive,SUITE 350,Las Vegas,NV,89119,United States,630
2101887,2120694,"American Residential Properties OP, L.P.",Foreign Limited Partnership,Delaware,NaN,Withdrawn,0,Real Estate and Rental and Leasing,630 Trade Center Drive,NaN,Las Vegas,NV,89119,United States,630


In [29]:
registry.loc[registry['StreetAddress1'] == '23975 Park Sorrento, Suite 300']

,BusinessId,BusinessName,BusinessTypeDesc,ForeignState,ForeignCountry,EntityStatus,Good Standing,NAICSCODE,StreetAddress1,StreetAddress2,City,State,Zip,Country,Adrno
1710107,1722791,"AH4R MANAGEMENT - GA, LLC",Foreign Limited Liability Company,Delaware,NaN,Active/Compliance,1,NaN,"23975 Park Sorrento, Suite 300",NaN,Calabasas,CA,91302,United States,23975
1750069,1763202,"RJ AMERICAN HOMES 4 RENT TWO, LLC",Foreign Limited Liability Company,Delaware,NaN,Active/Compliance,1,NaN,"23975 Park Sorrento, Suite 300",NaN,Calabasas,CA,91302,United States,23975
1858509,1871902,"AMERICAN HOMES 4 RENT TRS, LLC",Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,"23975 Park Sorrento, Suite 300",NaN,Calabasas,CA,91302,United States,23975
1961334,1974808,"AMH 2014-2 BORROWER, LP",Foreign Limited Partnership,Delaware,USA,Active/Compliance,1,NaN,"23975 Park Sorrento, Suite 300",NaN,Calabasas,CA,91302,United States,23975
1976070,1989554,"AMH 2014-3 EQUITY OWNER, LLC",Foreign Limited Liability Company,Delaware,USA,Active/Compliance,1,NaN,"23975 Park Sorrento, Suite 300",NaN,Calabasas,CA,91302,United States,23975
1999349,2012858,"AMERICAN HOMES 4 RENT, L.P.",Foreign Limited Partnership,Delaware,USA,Active/Compliance,1,NaN,"23975 Park Sorrento, Suite 300",`,Calabasas,CA,91302,United States,23975
2251144,2293400,"AMH Development, LLC",Foreign Limited Liability Company,Delaware,NaN,Active/Compliance,1,Any legal purpose,"23975 Park Sorrento, Suite 300",Suite 300,Calabasas,CA,91302,United States,23975
2414088,2484544,"DALLAS WALK HOMEOWNERS ASSOCIATION, INC.",Domestic Nonprofit Corporation,Georgia,NaN,Active/Compliance,1,Any legal purpose,"23975 Park Sorrento, Suite 300",NaN,Calabasas,CA,91302,United States,23975
2542993,2672389,"Hoke O'Kelly Mill Owners Association, Inc.",Domestic Nonprofit Corporation,Georgia,NaN,Active/Compliance,1,Any legal purpose,"23975 Park Sorrento, Suite 300",NaN,Calabasas,CA,91302,United States,23975
2650387,2799942,"AMH 2015-2 Borrower, LP",Foreign Limited Partnership,Delaware,NaN,Active/Compliance,1,Any legal purpose,"23975 Park Sorrento, Suite 300",NaN,Calabasas,CA,91302,United States,23975


### Finding: we need to heavily value address over name, but we cannot devalue name completely (since many businesses have the same address).

### Let's look at some examples from the actual sales data - see if we can find these corporations

Note: full_sales is actually just 2022 sales in this example by mistake

In [30]:
full_sales = pd.read_csv('../output/final_sales_parcel_merged.csv')

C:\Users\nicho\AppData\Local\Temp\ipykernel_14608\2767870617.py:1: DtypeWarning: Columns (0,1,2,3,4,8,28,31,43,45,48,51,53,74,75,77,80,84,91,93,95,141,144,155) have mixed types. Specify dtype option on import or set low_memory=False.
  full_sales = pd.read_csv('../output/final_sales_parcel_merged.csv')


In [31]:
len(full_sales.index)

478699

In [32]:
full_sales = full_sales[full_sales['Own1'].notna()]
full_sales = full_sales[full_sales['Owner Adrstr'].notna()]

In [33]:
len(full_sales.index)

426603

In [34]:
full_sales[full_sales['Own1'].str.contains("PROGRESS")][['Own1', 'Owner Adrno', 'Owner Adrstr']].sample(10)

,Own1,Owner Adrno,Owner Adrstr
430750,PROGRESS RESIDENTIAL BORROWER 13 LLC,NaN,P O BOX 4090
248153,PROGRESS RESIDENTIAL 2014 1 BORROWER LLC,6617.0,SCOTTSDALE
281979,PROGRESS RESIDENTIAL 2016 1 BORROWER LLC,NaN,P O BOX 4090
434069,PROGRESS RESIDENTIAL BORROWER 9 LLC,NaN,P O BOX 4090
48115,"PROGRESSIVE DEVELOPMENT ALLIANCE, LLC",595.0,PIEDMONT
432708,PROGRESS RESIDENTIAL BORROWER 13 LLC,NaN,P O BOX 4090
462761,PROGRESS ATLANTA LLC,NaN,PO BOX 4090
432133,PROGRESS RESIDENTIAL BORROWER 14 LLC,NaN,P O BOX 4090
376379,PROGRESS RESIDENTIAL BORROWER 4 LLC,NaN,PO BOX 4090
430007,PROGRESS RESIDENTIAL BORROWER 11 LLC,NaN,P O BOX 4090


In [35]:
full_sales[full_sales['Own1'].str.contains("AMERICAN HOMES")][['Own1', 'Owner Adrno', 'Owner Adrstr']].sample(10)

,Own1,Owner Adrno,Owner Adrstr
327807,SELECT AMERICAN HOMES LLC,950.0,EAGLES LANDING
257572,AMERICAN HOMES 4 RENT PROPERTIES EIGHT,30601.0,AGOURA
10164,RALFS AMERICAN HOMES LLC,100.0,GLENDALOUGH
278122,AMERICAN HOMES 4 RENT PROPERTIES EIGHT,30601.0,AGOURA
276288,AMERICAN HOMES 4 RENT PROPERTIES EIGHT,30601.0,AGOURA
278057,AMERICAN HOMES 4 RENT PROPERTIES EIGHT,30601.0,AGOURA
276849,AMERICAN HOMES 4 RENT PROPERTIES EIGHT,30601.0,AGOURA
290698,AMERICAN HOMES 4 RENT PROPERTIES EIGHT,30601.0,AGOURA
290862,AMERICAN HOMES 4 RENT PROPERTIES EIGHT,30601.0,AGOURA
327812,SELECT AMERICAN HOMES LLC,950.0,EAGLES LANDING


In [36]:
full_sales[full_sales['Own1'].str.contains("INVITATION")][['Own1', 'Owner Adrno', 'Owner Adrstr']]

,Own1,Owner Adrno,Owner Adrstr


In [37]:
full_sales[full_sales['Own1'].str.contains("INVITATION")][['Own1', 'Owner Adrno', 'Owner Adrstr']]

,Own1,Owner Adrno,Owner Adrstr


In [38]:
full_sales[full_sales['Own1'].str.contains("Invitation")][['Own1', 'Owner Adrno', 'Owner Adrstr']]

,Own1,Owner Adrno,Owner Adrstr


In [39]:
full_sales[full_sales['Own1'].str.contains("IH6")][['Own1', 'Owner Adrno', 'Owner Adrstr']]

,Own1,Owner Adrno,Owner Adrstr
303407,IH6 PROPERTY GEORGIA L P,8601.0,DUNWOODY
304200,IH6 PROPERTY GEORGIA LP,8601.0,DUNWOODY
305994,IH6 PROPERTY GEORGIA LP,8601.0,DUNWOODY
334587,IH6 PROPERTY GEORGIA LP,8601.0,DUNWOODY
478476,IH6 PROPERTY GEORGIA LP,1717.0,MAIN
478520,IH6 PROPERTY GEORGIA LP,1717.0,MAIN


In [40]:
registry.loc[registry['BusinessName'].str.contains("IH6")]

,BusinessId,BusinessName,BusinessTypeDesc,ForeignState,ForeignCountry,EntityStatus,Good Standing,NAICSCODE,StreetAddress1,StreetAddress2,City,State,Zip,Country,Adrno
2085240,2101704,"IH6 PROPERTY GEORGIA, L.P.",Foreign Limited Partnership,Delaware,NaN,Active/Compliance,1,Any legal purpose,1717 MAIN STREET,SUITE 2000,DALLAS,TX,75201,United States,1717


In [41]:
full_sales.groupby(['GRANTEE'])['GRANTEE'].count().sort_values(ascending=False).head(50)

GRANTEE
FEDERAL NATIONAL MORTGAGE ASSOCIATION       5226
BANK OF AMERICA N A                         2468
BAC HOME LOANS SERVICING LP                 2092
WELLS FARGO BANK NA                         1909
SECRETARY OF HOUSING & URBAN DEV            1783
FEDERAL HOME LOAN MORTGAGE CORPORATION      1727
FEDERAL NATIONAL MORTGAGE ASSN              1595
WELLS FARGO BANK N A                        1576
CITIMORTGAGE INC                            1467
SUNTRUST MORTGAGE INC                       1399
FEDERAL HOME LOAN MORTGAGE CORP             1271
FEDERAL NATL MTG ASSN                       1180
BANK OF NEW YORK MELLON THE                 1174
SECRETARY OF HOUSING & URBAN DEVELOPMENT    1079
CHASE HOME FINANCE LLC                       900
REDUS ATLANTA SUBDIVISIONS LLC               849
D R HORTON INC                               817
STATE BANK & TRUST COMPANY                   780
FEDERAL HOME LOAN MTG CORP                   749
STB LOTS LLC                                 725
CRM CENTRAL 

--- 

## ALGORITHM

Pre-processing:
- Create separate owner zip and owner address number columns for faster comparison.
- Lowercase all address and owner strings.

Naive Approach:
- Compare every entry of the sales data to every entry of the registry data; compare owner name and owner address separately and create a separate match score for each - this might result in a owner name that does not have the matched owner address...
- Take the closest match and create new columns "matched_owner" with owner name, address, and match score, based on business registry data
- Take only matches with high enough match score

Issues: 
- There are 3M entries in the business registry data and 50K entries in sales data. That is 3M*50K = 150B comparisons using a slow metric (Levenshtein or other)

Better Approach:
- Group sales data into Owner Zipcodes. Group registry data into Owner Zipcodes.
- Run the comparison between sales and registry data only when the zipcode matches. Only compare street name strings when adrno is the same. This is a much faster comparison (2 numbers than strings and no special similarity metric). It also reduces the search space; largest search space (zip code with most businesses) is 40K.
- Given the best match for owner address, search through the business names registered to that address for the top 2 similar owner names.
- Take the closest match (within zip code) and create new columns "matched_owner" with owner name, address, and match score, based on business registry data
- Take only matches with high enough match score
- Create a combined similarity metric at the end, using what we determine as best for coefficients for owner name similarity and owner address similarity.

If still not fast enough- use an Approximate Nearest Neighbor search / find a way to accurately vectorize.

In [48]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

In [43]:
matched_df = pd.DataFrame()

def find_closest(sale, registry_zip):
    match_score_adrr = 0
    match_address = None
    match_score_name = 0
    match_name = None

    for index, row in registry_zip.iterrows():
        if (sale['OWNER Adrno'] == sale['OWNER Adrno']) & (sale['OWNER Adrno'] == row['Adrno']):
            print("")
            print(sale['OWNER Adrstr'])
            print(row['StreetAddress1'])
            print("")
            ratio = fuzz.ratio(sale['OWNER Adrstr'].str.lower(), row['StreetAddress1'].str.lower())
            if ratio > match_score_adrr:
                match_score_adrr = ratio
                match_address = row['StreetAddress1']
    
    reduced = registry_zip[registry_zip['StreetAddress1'] == match_address]

    for index, row in reduced.iterrows():
        ratio = fuzz.ratio(sale['GRANTEE'].str.lower(), row['BusinessName'].str.lower())
        if ratio > match_score_name:
            match_score_name = ratio
            match_name = row['BusinessName']

    return match_address, match_score_adrr, match_name, match_score_name

def match_to_registry(sales, registry):
    matched = pd.DataFrame()
    first_flag = True

    for zip in sales['Zip1'].unique():
        sales_zip = sales[sales['Zip1'] == zip]
        registry_zip = registry[registry['Zip'] == zip]
        
        sales_zip[['match_address', 'match_score_adrr', 'match_name', 'match_score_name']] = \
            sales_zip.apply(lambda x: find_closest(x[['Zip1', 'OWNER Adrno', 'OWNER Adrstr', 'GRANTEE']], registry_zip), axis=1).tolist()
    
        if first_flag:
            first_flag = False
            matched = sales_zip
        else:
            matched = pd.concat([matched, sales_zip])

    return matched

In [54]:
def match_fuzz(df_1, df_2, key1, key2, limit=2):
    search = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, search, limit=limit))
    df_1['matches'] = m
    
    return df_1

In [61]:
sales_sub = full_sales[full_sales['Zip1'] == '30313']
registry_sub = registry[registry['Zip'] == '30313']

matched = match_fuzz(sales_sub, registry_sub, 'Own1', 'BusinessName')
matched['matches']

C:\Users\nicho\AppData\Local\Temp\ipykernel_14608\3461157588.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m


49251     [(LUTHER H. RANDALL III-OXFORD, LLC, 86), (TA ...
49252     [(LUTHER H. RANDALL III-OXFORD, LLC, 86), (TA ...
50860     [(REAL ESTATE SORS LLC, 86), (THURN & LINDBERG...
119129    [(Pristine Refinement by Christopher McKinley,...
119131    [(Pristine Refinement by Christopher McKinley,...
                                ...                        
446450    [(LOFTMAN CONSULTING, INC., 52), (PAUL RIECHMA...
446451    [(LOFTMAN CONSULTING, INC., 52), (PAUL RIECHMA...
446452    [(LOFTMAN CONSULTING, INC., 52), (PAUL RIECHMA...
446453    [(LOFTMAN CONSULTING, INC., 52), (PAUL RIECHMA...
446455    [(Ohm Salon and Spa LLC, 56), (OMEGA SALES, IN...
Name: matches, Length: 693, dtype: object

In [66]:
matched[['Own1', 'matches']].sample(10)

,Own1,matches
344357,JACKSON LEO J II & GWENDOLYN J,"[(J. LUCKEY, INC., 86), (BNV/RNU NORTHSIDE PHA..."
196324,KNIGHT FREDERICK C,"[(WINGNUTS RESTAURANTS, L.L.C., 86), (JOHNNY C..."
196071,MARTIN JASON D,"[(D.J. CONSTRUCTION/CAR DETAIL INC., 86), (K D..."
344337,OLIVIA J BYNUM TRUST THE,"[(J. LUCKEY, INC., 86), (THE S.T.L. (SKIES THE..."
198521,RUSSELL REALTY LIMITED PARTNERSHIP,"[(URBANSCAPE REALTY, INC., 86), (RUSSELL NEW Y..."
195291,TAGO FUMIAKI,"[(2ENGAGE FINANCIAL MEDIA, LLC, 52), (Maverick..."
195359,FARINHA GYASI,"[(SFOGARSI, INC., 53), (Antioch United Methodi..."
344688,SERYUS DEVELOPMENT LLC,"[(SERYUS DEVELOPMENT LLC, 100), (LUTHER H. RAN..."
196335,MC KINNON BONNY,"[(INFERNO MUSIC, INC., 53), (REEVES MACHINE CO..."
344657,CHAPPELL MELBREY E,"[(QUALITY R.E.A.L. SOLUTIONS, LLC, 86), (E. J...."


In [ ]:
full_sales[:100][['GRANTOR','GRANTEE','SALES PRICE', 'Own1', 'Owner Adrno', 'Owner Adrstr', 'Zip1', 'Owner Cityname']].head(2)